## Imswitch/UC2-REST DEMO for 8P370 CBL Microscopy

**By Marcus Vroemen and Tom van Hattem**

*Using the open UC2 Rest and Imswitch Software*

This tutorial and demo will shows you how to get started with Imswitch and UC2 in Python. Imswitch is a software package that allows the creation of graphical user interfaces (GUIs) and interact with our hardware, the open UC2 Controller Board to turn on LED matrices or control the motors. The board itself will not handle cameras, these will usually be handled over USB. Most online tutorials will show you. how to flash the boards. This should already be done on the controller boards, but if not they can easily be reflashed with help of this page: https://youseetoo.github.io/ <br>
But before reflashing your computer will require drivers to see the controller board. If the official openUC2 boards are used with a ESP32-WROOM-32D controller and shield, you will need the CP210x drivers: https://www.silabs.com/developers/usb-to-uart-bridge-vcp-drivers?tab=downloads
It is recommended to use Visual Studio Code or PyCharm, although Spyder is also possible. <br>

IMPORTANT: Do not attempt to use the motors without an external power supply. Powering over USB will most certainly not work/damage your computer.

*Sidenote: also ensure that you are using a data cable. Charging micro usb cables will not be able to interact with the controller board. <br>*

Let's get started. First we need to configure the python environment


### Python Environment Configuration




1. Create a new Python environment called `8P370`. Do this within the Anaconda command prompt, run:



```python
conda create --name 8P370 python=3.8
```

2. Make sure to activate the `8P370` environment before proceeding to install pacakges with the following command:


```python
conda activate 8P370
```



3. Next, install `UC2-REST` and `ImSwitch`  packages (and other packages you might need at the place of <...>) with `pip` using the following command:

```python
pip install UC2-REST 
pip install ImSwitchUC2==0.2.0.14
pip install <...> 
```

Hint: Make sure to only use `pip` from now on. Do not mix `pip` with `conda install`. Trust me.

###  basic Python Code to control the microscope

To use python code to control the microscope, we need to communicate with the UC2 Controller board. This is a USB connection through COM ports. In windows open the device manager, under Ports(COM&LPT) when the board is connected it should show `Silicon Labs CP210x USB to UART Bridge (COM4)`. COM4 could be a different port but by default it usually is 3 or 4. Change in the next code block the serialport to your COM port. It has to be a string.

In [1]:
import uc2rest as uc2

serialport = "COM76" # for Windows - change accordingly
#serialport = "/dev/cu.SLAB_USBtoUART" # for MAC change accordingly
#serialport = "/dev/cu.wchusbserial110" # for MAC change accordingly

ESP32 = uc2.UC2Client(serialport=serialport)

_state = ESP32.state.get_state()
print(_state)

ESP32.close()

2023-09-26 14:06:17 ERROR [UC2Client] string indices must be integers
2023-09-26 14:06:17 ERROR [UC2Client] string indices must be integers
2023-09-26 14:06:17 DEBUG [UC2Client] COM76
2023-09-26 14:06:17 DEBUG [UC2Client] Trying out port COM76 failed
2023-09-26 14:06:17 ERROR [UC2Client] could not open port 'COM76': PermissionError(13, 'Access is denied.', None, 5)
2023-09-26 14:06:17 DEBUG [UC2Client] No USB device connected! Using DUMMY!
2023-09-26 14:06:17 DEBUG [UC2Client] Using API version 2
2023-09-26 14:06:17 DEBUG [UC2Client] Attention, lasers are on channels 1,2,3


config is not valid



AttributeError: 'UC2Client' object has no attribute 'close'

In [7]:
import uc2rest
ESP32 = uc2rest.UC2Client(serialport="unknown")

# setting debug output of the serial to true - all message will be printed
ESP32.serial.DEBUG=False

2023-09-26 11:05:09 ERROR [UC2Client] could not open port 'unknown': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)
2023-09-26 11:05:09 DEBUG [UC2Client] COM76


2023-09-26 11:05:13 ERROR [UC2Client] string indices must be integers
2023-09-26 11:05:13 DEBUG [UC2Client] Trying out port COM76 failed
2023-09-26 11:05:13 ERROR [UC2Client] string indices must be integers
2023-09-26 11:05:13 DEBUG [UC2Client] No USB device connected! Using DUMMY!
2023-09-26 11:05:13 DEBUG [UC2Client] Using API version 2
2023-09-26 11:05:13 DEBUG [UC2Client] Attention, lasers are on channels 1,2,3


config is not valid


In [5]:
from ESP32Client import ESP32Client  
ESP32 = ESP32Client(serialport="unknown")

# move and measure
print("Current position: "+ str(ESP32.get_position(axis=1)))
ESP32.move_x(steps=1000, speed=1000, is_blocking=True, is_absolute=True, is_enabled=True)

# set a funny pattern
import numpy as np

Nx=8
Ny=8
led_pattern = np.abs(np.int8(np.random.randn(3,Nx*Ny)*255))
ESP32.send_LEDMatrix_array(led_pattern, timeout=1)

ModuleNotFoundError: No module named 'ESP32Client'

# TROUBLESHOOTING

There are several things that limit the board from being connected correctly to your laptop/pc. The first thing to check is if you have the correct drivers installed: https://www.silabs.com/developers/usb-to-uart-bridge-vcp-drivers?tab=downloads

Next check if you are using the correct cable, and if it shows up in the device manager. 

For the error could not open port 'COM3': PermissionError(13, 'Access is denied.', None, 5). It is likely that another program or kernel is running and has the com port opened. For example a program such as Ultimaker CURA or Arduino IDE is likely to interfere with UC2, therefore close it and disable opening the program on startup in the task manager (for convenience).

Also check that you are using the correct com port (as seen in the device manager.)

To check if the hardware works correctly it is also possible to check using https://youseetoo.github.io/indexWebSerialTest.html. This will ask in the browser to connect to the UC2 controller board and if a Z-stage motor or LED ring is correctly connected to the right port (labelled on the pcb) then you should be able to control them. 
